In [2]:
%load_ext autotimebar

Label(value='0s')

In [37]:
import matplotlib.pyplot as plt
import pandas as pd
import json
import numpy as np

%matplotlib inline

plt.style.use('seaborn-v0_8-dark-palette')

with open("ResultsJsons/results_22_06_2024__20_01_03.json", "r") as f:
    results = json.load(f)

maps = results["maps"]
possible_number_of_tasks = results["tasks_num"]
possible_number_of_task_freq = results["tasks_frequency"]
possible_number_of_agents = results["agents_num"]
possible_number_of_starts = results["start_num"]
possible_number_of_goals = results["goal_num"]

Label(value='0s')

In [46]:
def stats_of(map_dict):
    fixed = map_dict["fixed"]
    learning = map_dict["learning"]

    time = {
        "fixed": np.arange(0, len(fixed["costs"]), 1),
        "learning": np.arange(0, len(learning["costs"]), 1)
    }
    
    df_time_fixed = pd.DataFrame({
        "costs": fixed["costs"], "serv_times": fixed["serv_times"], "pickup_to_goal_times": fixed["pickup_to_goal_times"],
        "start_to_pickup_times": fixed["start_to_pickup_times"], "runtimes": fixed["runtimes"], "time": time["fixed"]
    }, index=time["fixed"])
    
    df_time_learning = pd.DataFrame({
        "costs": learning["costs"], "serv_times": learning["serv_times"], "pickup_to_goal_times": learning["pickup_to_goal_times"],
        "start_to_pickup_times": learning["start_to_pickup_times"], "runtimes": learning["runtimes"], "time": time["learning"]
    }, index=time["learning"])
    
    df_tasks_fixed = pd.DataFrame({
        "estimated": fixed["estimated_costs"], "real": fixed["real_costs"]
    })
    
    df_tasks_learning = pd.DataFrame({
        "estimated": learning["estimated_costs"], "real": learning["real_costs"]
    })
    
    assert fixed['agents'] == learning['agents'] and fixed['tasks'] == learning['tasks'] and fixed['map_name'] == learning[
        "map_name"] and fixed['task_frequency'] == learning['task_frequency'] and fixed['pickup'] == learning['pickup'] and \
           fixed['goal'] == learning['goal']
    
    config = {
        "agents": fixed["agents"], "tasks": fixed["tasks"], "map": fixed["map_name"],
        "pickup": fixed["pickup"], "goal": fixed["goal"], "task_freq": fixed["task_frequency"]
    }
    
    return config, df_time_fixed, df_time_learning, df_tasks_fixed, df_tasks_learning

for my_map in maps:
    print("Run_id:", my_map["run_id"])
    config, df_fixed, df_learning, df_fixed_costs, df_learning_costs = stats_of(my_map)
    print(config)
    print(df_fixed_costs)
    print(df_learning_costs)
    print(df_fixed)
    print(df_learning)


Label(value='0s')

Run_id: input_warehouse_mid_with_middle_corridors_agents_20_pickup_24_goal_14_tasks_80
{'agents': 20, 'tasks': 80, 'map': 'input_warehouse_mid_with_middle_corridors', 'pickup': 24, 'goal': 14, 'task_freq': 0.2}
        estimated  real
task1          17    17
task3          43    43
task2          25    25
task4           7     7
task5          46    46
...           ...   ...
task79         15    15
task76         23    23
task72         38    38
task78         46    46
task80         22    22

[80 rows x 2 columns]
        estimated  real
task2          12    13
task1          15    15
task3          21    23
task4          33    33
task5          17    17
...           ...   ...
task78         23    23
task77         66    66
task79         18    19
task76         54    54
task80         54    54

[80 rows x 2 columns]
     costs  serv_times  pickup_to_goal_times  start_to_pickup_times  runtimes  \
0        0    0.000000              0.000000               0.000000  0.034441   
1    